# De-gending text

Steps:
1. Load data for each different corpus into consistent format
    - Enron
    - X RtGender
2. X Train-Dev-Test Split on all of them
3. X Build TF-IDF vectorizors on all training datasets
4. X Build gender classifiers on all training datasets
5. Use a count-vectorizer to build the Delete Procedure
    - One vectorizer for each (male/female)
    - Use this to create a dict mapping each word to its count value
    - Lambda = 1; gamma = 5 (to start)
    - Then use a function to look up the count ratio for each word given a sentence
6. Make a similarity function - TF-IDF weighted word overlap between sentences.
    - Maybe pairwise pre-compute this for all combos?
7. Make Retrieve functions
    - First one that retrieves the most similar sentence in the target attribute
    - Next one that gets the top few sentences, and replaces the attributes in the source sentence one at a time with attributes in the target sentence(s)
    
Experiments:
1. Do each of these individually, end-to-end
2. Do each combination of applying different de-gendering systems to each other dataset. Apply the datasets trained gender classifier, and look at drop in acc, F1, and change in confusion matrix vs. before

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

In [ ]:
# Local imports
from dataset import Dataset

In [ ]:
PATH = 'data/rt_gender/'
ds = Dataset('fitocracy_responses', PATH)

In [ ]:
ds.report_clf_results()

In [ ]:
dev_pred_proba = ds.clf.predict_proba(ds.vecs.dev)

is_female = ds.labels.dev
is_male = 1 - ds.labels.dev

female_proba = dev_pred_proba[:, 1]
male_proba = dev_pred_proba[:, 0]

is_female_f_prob = [p for p, g in zip(female_proba, is_female)
                    if g]
is_female_m_prob = [p for p, g in zip(female_proba, is_male)
                    if g]

plt.hist([is_female_f_prob, is_female_m_prob], stacked=True)
plt.title(ds.name)
plt.show()

In [ ]:
# Fitocracy
plt.hist([is_female_f_prob, is_female_m_prob], stacked=True)
plt.show()

In [ ]:
ds.save_pickle()

# Now for Enron

In [ ]:
from collections import Counter

In [ ]:
name = 'enron'
PATH = 'data/vinod/'

In [ ]:
ds = Dataset('enron', PATH, text_col='main_text', sender_col='from_gender',
             recipient_col='to_gender')#, predict_on_sender=False,
             #sender_gender='M')

In [ ]:
ds.report_clf_results()

In [ ]:
dev_pred_proba = ds.clf.predict_proba(ds.vecs.dev)

is_female = ds.labels.dev
is_male = 1 - ds.labels.dev

female_proba = dev_pred_proba[:, 1]
male_proba = dev_pred_proba[:, 0]

is_female_f_prob = [p for p, g in zip(female_proba, is_female)
                    if g]
is_female_m_prob = [p for p, g in zip(female_proba, is_male)
                    if g]

plt.hist([is_female_f_prob, is_female_m_prob], stacked=True)
plt.show()

In [ ]:
ds.save_pickle()

In [ ]:
# How to load Datasets
ds = Dataset.load_pickle('data/final/enron_dataset.pkl')

# TODO
- X Get the counts for each word (naive-bayes counts)
    - Transform into logspace
- Set a gamma threshold
- For each sentence, split into context, attributes by this threshold

In [ ]:
from dataset import TransformedDataset

In [ ]:
t_dataset = TransformedDataset(ds, 3, 1)

In [ ]:
wc_df = t_dataset.wc_df

In [ ]:
wc_df.head()

In [ ]:
plt.hist(attrs, bins=50)
plt.show()

In [ ]:
t_dataset.male_splits.train.iloc[0]

# Evaluating baseline performance

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
train_pred = fitocracy.clf.predict(fitocracy.vecs.train)
dev_pred = fitocracy.clf.predict(fitocracy.vecs.dev)

In [ ]:
print(classification_report(fitocracy.labels.train, train_pred))

In [ ]:
print(classification_report(fitocracy.labels.dev, dev_pred))

# Building the count vectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
lambd = 1

In [ ]:
# Split male/female
male_df = fitocracy.df[fitocracy.df['responder_gender'] == 'M']
female_df = fitocracy.df[fitocracy.df['responder_gender'] == 'W']

In [ ]:
male_count_vectorizer = CountVectorizer(min_df=10).fit(male_df['response_text'])
female_count_vectorizer = CountVectorizer(min_df=10).fit(female_df['response_text'])

In [ ]:
m_word_counts = np.array(male_count_vectorizer.transform(
    [' '.join(male_df['response_text'])]).todense())[0]
f_word_counts = np.array(female_count_vectorizer.transform(
    [' '.join(female_df['response_text'])]).todense())[0]

In [ ]:
m_features = male_count_vectorizer.get_feature_names()
f_features = female_count_vectorizer.get_feature_names()

In [ ]:
f_df = pd.DataFrame(f_word_counts, index=f_features, columns=['female_count'])
m_df = pd.DataFrame(m_word_counts, index=m_features, columns=['male_count'])
full_df = pd.concat([f_df, m_df], axis=1)
full_df.fillna(0, inplace=True)
full_df['female_ratio'] = ((full_df['female_count'] + lambd) /
                    (full_df['male_count'] + lambd))
full_df['male_ratio'] = ((full_df['male_count'] + lambd) /
                    (full_df['female_count'] + lambd))

In [ ]:
word_set = set(full_df.index)

# Determining which words to replace

Given a sentence score each word, and if it is above gamma in favor of the current gender

In [ ]:
gamma = 5

In [ ]:
def score_words(row):
    text = row['response_text']
    gender = row['responder_gender']
    gender_col = 'female_ratio' if gender=='W' else 'male_ratio'
    
    vals = []
    
    for word in text.lower().split(' '):
        if word in word_set:
            val = full_df[gender_col][word]
            vals.append(val)
        else:
            vals.append(-1)
    return vals

In [ ]:
score_words(fitocracy.df.iloc[0])

In [ ]:
word_vecs = [score_words(row) for i, row in male_df[:10_000].iterrows()]

In [ ]:
full_df[full_df['male_ratio'] < 5]['male_ratio'].hist(bins=20)
plt.show()

# Replace some of these words

In [ ]:
m_df_subset = male_df[:10_000]

In [ ]:
# For now just sample a random word that biases male
sentences = [text.split(' ') for text in m_df_subset['response_text']]

In [ ]:
male_sampler = full_df[full_df['male_ratio'] > 1]
female_sampler = full_df[full_df['female_ratio'] > 1]

def get_random_word(gender):
    if gender == 'male':
        return male_sampler.sample(1).index[0]
    else:
        return female_sampler.sample(1).index[0]

In [ ]:
from tqdm import tqdm, tqdm_notebook

In [ ]:
n_words_seen = 0
n_words_changed = 0

for sent, vals in tqdm_notebook(zip(sentences, word_vecs)):
    for i, val in enumerate(vals):
        n_words_seen += 1
        if val > 1.2:
            n_words_changed += 1
            sent[i] = get_random_word('female')

In [ ]:
print(n_words_changed, '/', n_words_seen)

In [ ]:
print(n_words_changed, '/', n_words_seen)

In [ ]:
transformed_sentences = [' '.join(sent) for sent in sentences]

In [ ]:
transformed_sentences[:15]

# Now compare the prediction on these changes

In [ ]:
original_vecs = fitocracy.vectorizer.transform(m_df_subset['response_text'])
transformed_vecs = fitocracy.vectorizer.transform(transformed_sentences)

In [ ]:
true_labels = [0] * 10_000
orig_pred_labels = fitocracy.clf.predict(original_vecs)
trans_pred_labels = fitocracy.clf.predict(transformed_vecs)

In [ ]:
# Male
print(classification_report(true_labels, orig_pred_labels))
print(accuracy_score(true_labels, orig_pred_labels))

In [ ]:
# Male
print(classification_report(true_labels, trans_pred_labels))
print(accuracy_score(true_labels, trans_pred_labels))

In [ ]:
# Female
print(classification_report(true_labels, orig_pred_labels))
print(accuracy_score(true_labels, orig_pred_labels))

In [ ]:
# Female
print(classification_report(true_labels, trans_pred_labels))
print(accuracy_score(true_labels, trans_pred_labels))